# Zero to Hero

## 1.11 Arbol training/testing

El objetivo es calcular la ganancia en testing de un arbol de decision

In [2]:
rm( list=ls())
gc()

library( "data.table")   #cargo la libreria  data.table
library( "rpart")  #cargo la libreria  rpart
library( "rpart.plot")
library("caret")

options(repr.plot.width=20, repr.plot.height=10) 
setwd("/Users/clara/Documents/00-Posgrado/4_DM_Eco_y_Finanzas/")  #Aqui se debe poner la ruta de la PC local

,used,(Mb),gc trigger,(Mb),limit (Mb),max used,(Mb)
Ncells,580048,31.0,1247334,66.7,NA,1156650,61.8
Vcells,1090860,8.4,8388608,64.0,16384,1823845,14.0


Loading required package: lattice

Loading required package: ggplot2



In [7]:
dataset <- data.table::fread("./datasetsOri/paquete_premium_202011.csv")   #cargo el dataset
head(dataset)

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<lgl>
4572294,202011,1,0,0,46,91,1829.57,8776.24,330.96,⋯,5,0,-30785.09,22.89,2752,1966.41,4,0,8318.7,NA
4572366,202011,1,0,0,48,200,-810.10,39016.56,293.35,⋯,5,0,-25389.00,0.00,3424,2473.26,5,0,18263.7,NA
4572633,202011,1,0,0,56,126,28081.94,350084.12,3037.83,⋯,5,0,-92330.79,0.00,3767,49619.75,15,0,4527.9,NA
4572728,202011,1,0,0,56,274,-348.43,6001.98,200.19,⋯,5,0,-19574.32,0.00,5013,21451.54,6,0,1041.3,NA
4572765,202011,1,0,0,50,301,3940.74,34861.95,2609.04,⋯,5,0,-18382.02,0.00,2127,0.00,0,0,1684.8,NA
4572913,202011,1,0,0,46,271,11845.15,110847.64,1360.30,⋯,5,0,-5905.90,0.00,2001,2289.44,2,0,1544.4,NA


In [8]:
#creo training con 70%  y testing con 30%
set.seed(13)
train_rows <- createDataPartition(dataset$clase_ternaria, p= 0.70,list= FALSE)
dtrain  <- dataset[ train_rows]
dtest  <-  dataset[ -train_rows]

Entreno el modelo en los datos de training

In [10]:
param  <- list("cp"= -0.5,
               "minsplit"=  900,
               "minbucket"= 440,
               "maxdepth"= 5
              )

Entreno el modelo en los datos de training

In [11]:
#genero el modelo
modelo <-  rpart::rpart(formula= "clase_ternaria ~ ." ,
                        data= dtrain,
                        xval= 0,
                        control= param)

aplico el modelo a los datos de testing

In [12]:
prediccion  <- predict( modelo, dtest, type = "prob")

ERROR: Error in predict.rpart(modelo, dtest, type = "prob"): Invalid prediction for "rpart" object


### Calculo la ganancia del modelo en los datos de testing

a partir de la prediccion, calculo la probabilidad de BAJA+2 de cada registro de testing  **dtest**

In [13]:
prob_baja2  <- prediccion[, "BAJA+2"]

ERROR: Error in eval(expr, envir, enclos): object 'prediccion' not found


Para el cálculo de la ganancia del modelo aplicado a testing debo tener en cuenta lo siguiente
* Solo envio estimulo a los registros que el modelo asigno una probabilidad mayor a 0.025
* Si no envio estimulo, no gano ni pierdo, es decir la ganacia es cero
* Si el registro al que envio el estímulo es un **BAJA+2**  entonces la ganancia es de +48750
* Si el registro al que envio el estímulo es un **BAJA+1**  entonces la ganancia es de  -1250
* Si el registro al que envio el estímulo es un **CONTINUA** entonces la ganancia es de  -1250

A la condición que solo envio a los registros con probabilidad mayor a 0.025 la resuelvo con la condicion lógica  (prob_baja2>0.025)
<br>
Esta condicion termina siendo un 0 o un 1
<br>
vale 1 si y solo si se cumple la condicion, es decir  si prob_baja2 es mayor a 0.025

La ganancia de +48750 o -1250 la resuelvo de esta forma ifelse( clase_ternaria=="BAJA+2", 48750, -1250)

Finalmente,  que se cumplan ambas condiciones al mismo tiempo, en este caso, es multiplicarlas, o sea
<br>
(prob_baja2>0.025) * ifelse( clase_ternaria=="BAJA+2", 48750, -1250)
<br>
La condicion anterior toma los siguientes valores

| prob(BAJA+2) | clase_ternaria | ganancia |
| --- | :---: | ---: |
| menor o igual a 0.025 | BAJA+1 | 0 |
| menor o igual a 0.025 | CONTINUA | 0 |
| menor o igual a 0.025 | BAJA+2 | 0 |
| mayor a 0.025 | BAJA+1 | -1250 |
| mayor a 0.025 | CONTINUA | -1250 |
| mayor a 0.025 | BAJA+2 | 48750 |


Finalmente hago el calculo en data.table

In [8]:
ganancia_testing <- dtest[ , sum(  (prob_baja2>0.025) * ifelse( clase_ternaria=="BAJA+2", 48750, -1250) )]

In [9]:
ganancia_testing

[1] 2142500

Es importante que la ganancia esta calculada sobre los datos de testing, que en este caso son apenas el 30%
<br>
Si quiero extrapolar a todo el dataset, debo hacer el cociente de esa ganancia por 0.30

In [10]:
ganancia_testing_normalizada  <-  ganancia_testing/0.3

In [11]:
ganancia_testing_normalizada

[1] 7141667